# **1. Libraries**

## **1.1 Requirements:**

In [66]:
%pip install pandas sentence-transformers requests google-generativeai openai

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


## **1.2. Imports:**

In [1]:
import pandas as pd
from sentence_transformers import SentenceTransformer, util
import requests
import json
import google.generativeai as genai
import openai
import warnings
warnings.filterwarnings('ignore')

/Users/slondono/Documents/Cursos/Bootcamp/Proyecto/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/slondono/Documents/Cursos/Bootcamp/Proyecto/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# **2. Dataset**

In [2]:
def load_motorcycle_dataset():
    try:
        print("Cargando dataset de motocicletas desde archivo local...")
        df = pd.read_csv("data/all_bikez_curated.csv")  
        print(f"Dataset cargado: {len(df)} registros")
        return df
    except Exception as e:
        print(f"Error cargando dataset local: {e}")

df = load_motorcycle_dataset()

Cargando dataset de motocicletas desde archivo local...
Dataset cargado: 38472 registros


In [3]:
df.head()   

,Brand,Model,Year,Category,Rating,Displacement (ccm),Power (hp),Torque (Nm),Engine cylinder,Engine stroke,...,Dry weight (kg),Wheelbase (mm),Seat height (mm),Front brakes,Rear brakes,Front tire,Rear tire,Front suspension,Rear suspension,Color options
0,acabion,da vinci 650-vi,2011,Prototype / concept model,3.2,NaN,804.0,NaN,Electric,Electric,...,420.0,NaN,NaN,Single disc,Single disc,NaN,NaN,NaN,NaN,NaN
1,acabion,gtbo 55,2007,Sport,2.6,1300.0,541.0,420.0,In-line four,four-stroke,...,360.0,NaN,NaN,Bajaj,NaN,NaN,NaN,NaN,NaN,NaN
2,acabion,gtbo 600 daytona-vi,2011,Prototype / concept model,3.5,NaN,536.0,NaN,Electric,Electric,...,420.0,NaN,NaN,Single disc,Single disc,NaN,NaN,NaN,NaN,NaN
3,acabion,gtbo 600 daytona-vi,2021,Prototype / concept model,NaN,NaN,536.0,NaN,Electric,Electric,...,420.0,NaN,NaN,Single disc,Single disc,NaN,NaN,NaN,NaN,NaN
4,acabion,gtbo 70,2007,Prototype / concept model,3.1,1300.0,689.0,490.0,In-line four,four-stroke,...,300.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Custom made.


# **3. Embeddings**

In [4]:
df['full_name'] = df['Brand'].astype(str) + ' ' + df['Model'].astype(str) + ' ' + df['Year'].astype(str) +  ' ' + df['Category'].astype(str)

model = SentenceTransformer('all-MiniLM-L6-v2')
bike_embeddings = model.encode(df['full_name'].tolist(), convert_to_tensor=True)

print(f"Embeddings generados para {len(df)} motos")

Embeddings generados para 38472 motos


In [9]:
df['full_name'].head()

0    acabion da vinci 650-vi 2011 Prototype / conce...
1                           acabion gtbo 55 2007 Sport
2    acabion gtbo 600 daytona-vi 2011 Prototype / c...
3    acabion gtbo 600 daytona-vi 2021 Prototype / c...
4       acabion gtbo 70 2007 Prototype / concept model
Name: full_name, dtype: object

In [10]:
bike_embeddings

tensor([[-0.1338, -0.0079, -0.0340,  ..., -0.0317, -0.0534,  0.0324],
        [-0.0895,  0.0291, -0.1056,  ..., -0.0358, -0.0387,  0.0156],
        [-0.1291,  0.0184, -0.0784,  ..., -0.0323, -0.0505,  0.0279],
        ...,
        [-0.0672,  0.1617, -0.1409,  ..., -0.0288, -0.0390,  0.0409],
        [-0.1274,  0.1636, -0.1557,  ..., -0.0287, -0.0322,  0.0429],
        [-0.1304,  0.1540, -0.1591,  ..., -0.0246, -0.0394,  0.0234]],
       device='mps:0')

# **4. Models**

In [11]:
def preparar_consulta(consulta, threshold=0.6, top_n=100):
    print(f"Procesando consulta: '{consulta}'")

    query_embedding = model.encode(consulta, convert_to_tensor=True)
    scores = util.cos_sim(query_embedding, bike_embeddings)[0]
    best_idx = scores.argmax().item()
    best_score = scores[best_idx].item()

    print(f"Mejor coincidencia embedding: {best_score:.3f}")
    if best_score >= threshold:
        print(f"   -> {df.iloc[best_idx]['Brand']} {df.iloc[best_idx]['Model']}")

    top_indices = scores.topk(top_n).indices.tolist()

    catalogo_str = ""
    for i, idx in enumerate(top_indices):
        row = df.iloc[idx]
        catalogo_str += f"{i+1}. {row['Brand']} {row['Model']}"
        if 'Displacement (ccm)' in row and pd.notna(row['Displacement (ccm)']):
            catalogo_str += f" {int(row['Displacement (ccm)'])}cc"
        if 'Year' in row and pd.notna(row['Year']):
            catalogo_str += f" ({int(row['Year'])})"
        if 'Category' in row and pd.notna(row['Category']):
            catalogo_str += f" - {row['Category']}"
        catalogo_str += "\n"

    return catalogo_str, best_score


In [12]:
def generar_prompt(consulta, catalogo_str):
    return (
        "Eres experto en motocicletas. Identifica la moto exacta basándote en la consulta no estructurada.\n\n"
        f"{catalogo_str}\n"
        f'CONSULTA: "{consulta}"\n\n'
        "REGLAS:\n"
        '- "BAJ" o "BAJA" = Bajaj\n'
        '- "DISC" o "DISCO" = Discover\n'
        "- Números pueden ser cilindraje o año\n"
        "- Corrige errores de escritura\n"
        "- Elige la coincidencia más probable\n\n"
        "RESPONDE SOLO JSON VÁLIDO SIN MARKDOWN:\n"
        "{{\n"
        '  "marca": "Marca exacta",\n'
        '  "modelo": "Modelo exacto",\n'
        '  "cilindraje": "XXX o null",\n'
        '  "año": "XXXX o null",\n'
        '  "categoria": "Categoría o null",\n'
        '  "confianza": 0.95,\n'
        '  "razonamiento": "Explicación breve"\n'
        "}}"
    )


In [13]:
def limpiar_respuesta_json(respuesta_texto):

    if "```json" in respuesta_texto:
        respuesta_texto = respuesta_texto.split("```json")[1].split("```")[0].strip()
    elif "```" in respuesta_texto:
        respuesta_texto = respuesta_texto.split("```")[1].strip()
    try:
        return json.loads(respuesta_texto)
    except json.JSONDecodeError:
        import re
        json_match = re.search(r'\{.*?\}', respuesta_texto, re.DOTALL)
        if json_match:
            try:
                return json.loads(json_match.group())
            except json.JSONDecodeError:
                pass
        
        return {"error": "Respuesta no es JSON válido", "raw": respuesta_texto}

## **4.1. DeepSeek:**

In [14]:
class DeepSeekAdapter:
    def __init__(self, api_key):
        self.api_key = api_key
        self.name = "DeepSeek"
    
    def llamar_llm(self, prompt):
        try:
            print(f"Llamando a DeepSeek con prompt: {prompt}")  
            response = requests.post(
                "https://api.deepseek.com/v1/chat/completions",
                headers={
                    "Authorization": f"Bearer {self.api_key}",
                    "Content-Type": "application/json"
                },
                json={
                    "model": "deepseek-chat",
                    "messages": [{"role": "user", "content": prompt}],
                    "max_tokens": 300,
                    "temperature": 0.1
                },
                timeout=30
            )
            
            if response.status_code == 200:
                return response.json()["choices"][0]["message"]["content"]
            else:
                return {"error": f"API Error {response.status_code}"}
                
        except Exception as e:
            return {"error": f"Error conexión DeepSeek: {str(e)}"}

## **4.2. Gemini:**

In [15]:
class GeminiAdapter:
    def __init__(self, api_key, model_name="gemini-2.0-flash"):
        self.api_key = api_key
        self.model_name = model_name
        self.name = "Gemini"
        genai.configure(api_key=api_key)
        self.model = genai.GenerativeModel(model_name)
    
    def llamar_llm(self, prompt):
        try:
            generation_config = genai.types.GenerationConfig(
                temperature=0.1,
                max_output_tokens=300,
            )
            
            response = self.model.generate_content(
                prompt,
                generation_config=generation_config
            )
            
            return response.text
            
        except Exception as e:
            return {"error": f"Error conexión Gemini: {str(e)}"}

# 5. Testing

In [ ]:
API_KEYS = {
    "deepseek": "xxxxxxxxxx-xxxx-xxxx-xxxx-xxxxxxxxxxxx",
    "gemini": "xxxxxxxxxx-xxxx-xxxx-xxxx-xxxxxxxxxxxx"
}

In [17]:
def buscar_moto_universal(consulta, llm_adapter):
    catalogo_str, best_score = preparar_consulta(consulta)
    
    if best_score < 0.3:
        return {"error": "Consulta no coincide con ninguna moto en el catálogo"}
    
    prompt = generar_prompt(consulta, catalogo_str)
    
    print(f"Enviando prompt al modelo {llm_adapter.name}...")
    
    respuesta_texto = llm_adapter.llamar_llm(prompt)
    
    if isinstance(respuesta_texto, dict) and "error" in respuesta_texto:
        return respuesta_texto
    
    print("Respuesta recibida del modelo.")
    
    resultado_json = limpiar_respuesta_json(respuesta_texto)
    
    if "error" in resultado_json:
        return resultado_json
    
    return resultado_json

In [18]:
def ejecutar_casos_prueba_universal(llm_adapter):
    casos_prueba = [
        {"input": "BAJ DISC 125 2015", "esperado": "Bajaj Discover 125 2015"},
        {"input": "DISCO 125", "esperado": "Bajaj Discover 125"},
        {"input": "DISCOVER 126", "esperado": "Bajaj Discover 126"},
        {"input": "125ST 15'", "esperado": "Motocicleta 125cc del 2015"},
        {"input": "FAZER 125", "esperado": "Yamaha Fazer 125"},
        {"input": "FZ 125", "esperado": "Yamaha FZ 125"},
        {"input": "YAMAHA FZ z", "esperado": "Yamaha FZ 125"},
    ]
    
    print(f"INICIANDO PRUEBAS CON {llm_adapter.name}")
    print("=" * 60)
    
    resultados = []
    
    for i, caso in enumerate(casos_prueba, 1):
        print(f"\n[{i}/{len(casos_prueba)}] PROCESANDO CASO")
        print("=" * 50)
        print(f"INPUT: '{caso['input']}'")
        print(f"ESPERADO: {caso['esperado']}")
        print("-" * 30)
        
        try:
            resultado = buscar_moto_universal(caso['input'], llm_adapter)
            
            if "error" not in resultado:
                print("RESULTADO EXITOSO:")
                print(f"  Marca: {resultado.get('marca', 'N/A')}")
                print(f"  Modelo: {resultado.get('modelo', 'N/A')}")
                print(f"  Cilindraje: {resultado.get('cilindraje', 'N/A')}")
                print(f"  Año: {resultado.get('año', 'N/A')}")
                print(f"  Categoría: {resultado.get('categoria', 'N/A')}")
                print(f"  Confianza: {resultado.get('confianza', 0):.2f}")
                print(f"  Razonamiento: {resultado.get('razonamiento', 'N/A')}")
            else:
                print(f"ERROR: {resultado['error']}")
                if 'raw' in resultado:
                    print(f"Respuesta cruda: {resultado['raw'][:200]}...")
            
            resultados.append({
                'caso_num': i,
                'input': caso['input'],
                'esperado': caso['esperado'],
                'resultado': resultado,
                'modelo': llm_adapter.name,
                'timestamp': pd.Timestamp.now()
            })
            
        except Exception as e:
            print(f"EXCEPCIÓN: {str(e)}")
            resultados.append({
                'caso_num': i,
                'input': caso['input'],
                'esperado': caso['esperado'],
                'resultado': {'error': f'Excepción: {str(e)}'},
                'modelo': llm_adapter.name,
                'timestamp': pd.Timestamp.now()
            })
        
        print("-" * 50)
    
    return resultados

In [19]:
def comparar_modelos(modelos_disponibles):
    todos_resultados = {}
    
    for nombre, adapter in modelos_disponibles.items():
        if adapter is not None:
            print(f"\n{'='*80}")
            print(f"PROBANDO MODELO: {nombre.upper()}")
            print(f"{'='*80}")
            
            resultados = ejecutar_casos_prueba_universal(adapter)
            todos_resultados[nombre] = resultados
        else:
            print(f"\nSaltando {nombre} - API key no configurada")
    
    return todos_resultados

In [28]:
def exportar_resultados_comparacion(resultados_comparacion, archivo="salida.txt"):
    def safe_str(val, default="N/A"):
        return str(val) if val is not None else default

    with open(archivo, "w", encoding="utf-8") as f:
        for modelo, resultados in resultados_comparacion.items():
            f.write("#" * 80 + "\n")
            f.write(f"RESULTADOS DEL MODELO: {modelo.upper()}\n")
            f.write("#" * 80 + "\n\n")

            for res in resultados:
                r = res["resultado"]
                f.write("=" * 60 + "\n")
                f.write(f"Caso #{res['caso_num']} — Modelo: {modelo}\n")
                f.write(f"Entrada: {res['input']}\n")
                f.write(f"Esperado: {res['esperado']}\n\n")

                if "error" in r:
                    f.write(f"ERROR: {r['error']}\n")
                    if 'raw' in r:
                        f.write(f"Respuesta cruda: {r['raw'][:200]}...\n")
                else:
                    f.write("RESULTADO DEL MODELO:\n\n")
                    f.write("┌────────────────┬──────────────────────────────┐\n")
                    f.write(f"│ {'Marca':<16}│ {safe_str(r.get('marca')):<28}│\n")
                    f.write(f"│ {'Modelo':<16}│ {safe_str(r.get('modelo')):<28}│\n")
                    f.write(f"│ {'Cilindraje':<16}│ {safe_str(r.get('cilindraje')):<28}│\n")
                    f.write(f"│ {'Año':<16}│ {safe_str(r.get('año')):<28}│\n")
                    f.write(f"│ {'Categoría':<16}│ {safe_str(r.get('categoria')):<28}│\n")
                    f.write(f"│ {'Confianza':<16}│ {r.get('confianza', 0.0):<28.2f}│\n")
                    f.write("└────────────────┴──────────────────────────────┘\n\n")
                    f.write("Razonamiento:\n" + safe_str(r.get('razonamiento')) + "\n")
                f.write("=" * 60 + "\n\n")

    print(f"\nResultados exportados a '{archivo}'")


In [20]:
modelos_disponibles = {}    
# DeepSeek
modelos_disponibles["deepseek"] = DeepSeekAdapter(API_KEYS["deepseek"])
# Gemini
modelos_disponibles["gemini"] = GeminiAdapter(API_KEYS["gemini"])
    
print("Modelos disponibles:", list(modelos_disponibles.keys()))
    
# Opción 1: Probar un modelo específico
# if "deepseek" in modelos_disponibles:
#     resultados_deepseek = ejecutar_casos_prueba_universal(modelos_disponibles["deepseek"])

# if "gemini" in modelos_disponibles:
#     print("\nProbando solo Gemini:")
#     resultados_gemini = ejecutar_casos_prueba_universal(modelos_disponibles["gemini"])

# Opción 2: Comparar múltiples modelos
print("\nComparando todos los modelos disponibles:")
resultados_comparacion = comparar_modelos(modelos_disponibles)


Modelos disponibles: ['deepseek', 'gemini']

Comparando todos los modelos disponibles:

PROBANDO MODELO: DEEPSEEK
INICIANDO PRUEBAS CON DeepSeek

[1/7] PROCESANDO CASO
INPUT: 'BAJ DISC 125 2015'
ESPERADO: Bajaj Discover 125 2015
------------------------------
Procesando consulta: 'BAJ DISC 125 2015'
Mejor coincidencia embedding: 0.547
Enviando prompt al modelo DeepSeek...
Llamando a DeepSeek con prompt: Eres experto en motocicletas. Identifica la moto exacta basándote en la consulta no estructurada.

1. chang-jiang bd 125 157cc (2007) - Classic
2. chang-jiang bd 125-2b 157cc (2007) - Classic
3. chang-jiang bd 125-5a 157cc (2007) - Classic
4. chang-jiang bd 125-2 157cc (2007) - Classic
5. bajaj platina 125 dts-si 124cc (2010) - Classic
6. bajaj platina 125 dts-si 124cc (2009) - Classic
7. qjmotor qj500-8a 500cc (2021) - Classic
8. borile b500cr 487cc (2015) - Classic
9. qjmotor src500 530cc (2022) - Classic
10. jawa-cz 125 124cc (2013) - Sport
11. kymco pulsar 125 124cc (2008) - Allroun

In [29]:
exportar_resultados_comparacion(resultados_comparacion)


Resultados exportados a 'salida.txt'
